Look sometimes my response to stress is to code ok

In [102]:
.ORIG x3000

;; Actual code starts at START, after stack setup
    LEA R0, START
    JMP R0

;; R6 will store "going" addresses while going to push/pop
;; R7 will store "return" addresses while going to push/pop
;; R5 will be used in push/pop to point to top of stack, so won't be saved

;; To call a subroutine: 
;;     LEA R6, [SUBROUTINE LABEL]
;;     JSR PUSH

;; Before returning from a subroutine: 
;;     LEA R0, POP
;;     JMP R0

;; Path: 
;; Store address of subroutine in R6
;; JSR to push, which stores starting point in R7
;; Push does JMP R6 to get to subroutine
;; Subroutine JMPs to pop
;; Pop RETs to starting point 

;; Eventually, we may need to alter some of these instructions as things get too far apart for JSR or w/e.

;; Setup for the stack 
Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        .FILL xC180         ;; This is JMP R6, the assembler just wouldn't recognize it if I typed it out, for Mystery Reasons.
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET
        
START:  AND R0, R0, #0
        ADD R0, R0, #10 
        ST R0, N            ;; Put 10 into N, which TRI subroutine will use as input
        
        LEA R6, TRI
        JSR PUSH
        
        LD R2, TRI_RESULT   ;; Load TRI_RESULT so that it can be seen easily in regs
        HALT     
     
;; Subroutine that calculates triangle numbers - R0 must first be cleared, n is R3, sum is R0

TRI:    LD R0, TRI_RESULT   ;; Load TRI_RESULT and N into R0 and R3
        LD R3, N
        ADD R0, R0, R3      ;; Add R3 to R0 and decrement R0
        ADD R3, R3, #-1
        ST R0, TRI_RESULT   ;; Store those values back into TRI_RESULT and N
        ST R3, N

        ADD R3, R3, #0
        BRz DONE            ;; Base case: R3 (N) == 0
        
        LEA R6, TRI
        JSR PUSH
        BRnzp DONE
        
TRI_RESULT: .FILL #0
N:      .FILL #0
        
DONE:   LEA R0, POP
        JMP R0
.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [103]:
%exe

Computation completed
Instructions: 476
Cycles: 3646 (0.001823 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x000A R1: x0000 R2: x0037 R3: x0000 
R4: x0000 R5: x3003 R6: x3090 R7: x3090 


TRI_RESULT was loaded into R2. And indeed x37 = 3 * 16 + 7 = 55 = 10th triangle number. 